# Clipping based on lat/long

Based on this Medium post: https://medium.com/towards-data-science/youve-got-a-time-series-now-what-38803e079175

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
import xarray as xr

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import scipy

import os

# Import raster

In [4]:
#setwd for rasters
os.chdir('/Users/ecompu/Desktop/Climate/Maps')

# landcover22 = xr.open_dataset('satellite_land_cover_2022.nc')
# landcover21 = xr.open_dataset('satellite_land_cover_2021.nc')
# landcover20 = xr.open_dataset('satellite_land_cover_2020.nc')
landcover19 = xr.open_dataset('satellite_land_cover_2019.nc')
# landcover18 = xr.open_dataset('satellite_land_cover_2018.nc')
# landcover17 = xr.open_dataset('satellite_land_cover_2017.nc')

landcover19

<xarray.Dataset> Size: 101GB
Dimensions:              (time: 1, lat: 64800, lon: 129600, bounds: 2)
Coordinates:
  * lat                  (lat) float64 518kB 90.0 90.0 89.99 ... -90.0 -90.0
  * lon                  (lon) float64 1MB -180.0 -180.0 -180.0 ... 180.0 180.0
  * time                 (time) datetime64[ns] 8B 2019-01-01
Dimensions without coordinates: bounds
Data variables:
    lccs_class           (time, lat, lon) uint8 8GB ...
    processed_flag       (time, lat, lon) float32 34GB ...
    current_pixel_state  (time, lat, lon) float32 34GB ...
    observation_count    (time, lat, lon) uint16 17GB ...
    change_count         (time, lat, lon) uint8 8GB ...
    crs                  int32 4B ...
    lat_bounds           (lat, bounds) float64 1MB ...
    lon_bounds           (lon, bounds) float64 2MB ...
    time_bounds          (time, bounds) datetime64[ns] 16B ...
Attributes: (12/38)
    title:                      Land Cover Map of 2019
    summary:                    This dataset characterizes the land cover of ...
    type:                       C3S-LC-L4-LCCS-Map-300m-P1Y
    references:                 https://cds.climate.copernicus.eu/
    institution:                UCLouvain
    contact:                    copernicus-support@ecmwf.int
    ...                         ...
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.002778
    id:                         C3S-LC-L4-LCCS-Map-300m-P1Y-2019-v2.1.1
    project:                    EC C3S Land Cover
    geospatial_lon_min:         -180.0
    geospatial_lon_max:         180.0

# Import georeferenced csv files for countries of interest

In [3]:
os.chdir('/Users/ecompu/Desktop/Climate/georeferenced')

countries = ['kenya','tanzania','mozambique','uganda']

#load csv files for the four countries. File name examples is kenya_georef.csv
kenya_georef = pd.read_csv('kenya_georef.csv')
tanzania_georef = pd.read_csv('tanzania_georef.csv')
mozambique_georef = pd.read_csv('mozambique_georef.csv')
uganda_georef = pd.read_csv('uganda_georef.csv')

#concatenate all files into a single georef file
georef = pd.concat([kenya_georef,tanzania_georef,mozambique_georef,uganda_georef])

#change names of latitude and Longitude to match the names in the landcover files
georef.rename(columns={'Latitude':'lat','Longitude':'lon'},inplace=True)




Check that max and min latitudes are within the latitudes we want

In [4]:
# georef.head()
print(georef['lon'].min())
print(georef['lon'].max())
print(georef['lat'].min())
print(georef['lat'].max())

29.6591676
40.9010641
-26.45969795
3.5351114


Check that all points are within our countries of interest

In [5]:
# georef = georef.dropna(subset=['lat', 'lon'])

# # plot georef points in folium map
# import folium

# m = folium.Map(location=[-1.286389, 36.817223], zoom_start=6)

# for i in range(0,len(georef)):
#     folium.Marker([georef.iloc[i]['lat'], georef.iloc[i]['lon']], popup=georef.iloc[i]['lat']).add_to(m)
# m

# Slice the raster based on the georef file

In [6]:
#Slice the raster based on the georef file.
# landcover22 = landcover22.sel(lon=slice(georef['lon'].min(),georef['lon'].max()),lat=slice(georef['lat'].max(),georef['lat'].min()))
# landcover21 = landcover21.sel(lon=slice(georef['lon'].min(),georef['lon'].max()),lat=slice(georef['lat'].max(),georef['lat'].min()))
landcover20 = landcover20.sel(lon=slice(georef['lon'].min(),georef['lon'].max()),lat=slice(georef['lat'].max(),georef['lat'].min()))





Ideally, each of the land cover files should be joined to the georef file, based on latitude and longitude, so they can then be joined by city name to the Afrobarometer data. Because of time constraints of the project, this step was not carried out, but feel free to uncomment the following chunk of code to join it. 

In [7]:
#join georef and landcover
# Extract the 'lccs_class' variable from the xarray Dataset
# lccs_class = landcover22['lccs_class'].squeeze('time')  # Remove the time dimension

# # Join the DataFrame with the xarray Dataset based on lat/lon
# georef['lccs_class'] = lccs_class.sel(lat=xr.DataArray(georef['lat']), 
#                                       lon=xr.DataArray(georef['lon']),
#                                       method='nearest').values

# georef

In [11]:
os.chdir('/Users/ecompu/Desktop/Climate/csv')


# landcover22df = landcover22.to_dataframe().to_csv('land_cover_22.csv')
# landcover21df = landcover21.to_dataframe().to_csv('land_cover_21.csv')
# landcover20df = landcover20.to_dataframe().to_csv('land_cover_20.csv')

: 

In [9]:
# To open it in QGIS, save as clipped nc file. 
os.chdir('/Users/ecompu/Desktop/Climate/Clipped_files')

# landcover22.to_netcdf('landcover22_countries.nc')
# landcover21.to_netcdf('landcover21_countries.nc')
# landcover20.to_netcdf('landcover20_countries.nc')
